In [1]:
import os
import pandas as pd
import numpy as np
import glob
from PIL import Image

import matplotlib.pyplot as plt

import json
from deepface.extendedmodels import Age, Gender, Race, Emotion
from deepface import DeepFace

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from os import path
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
import sys

import random
import pickle

In [2]:
with open('./FV/FValign/FV_personlist.pkl','rb') as f:
    Face2Voice = pickle.load(f)

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# train
testFace_path = './Face/DATA/sample10_5type/train/'
testVoice_path = './Voice/data/LibriSpeech/data_split/'

#매칭해준 사람에 맞춰 각 modality의 data align  
align_result = {}
for f,v in zip(Face2Voice, Face2Voice.values()):
    tempFace = os.listdir(testFace_path + f)
    #voice가 female에 있으면
    if v in os.listdir(testVoice_path + '0/train'):
        tempVoice = os.listdir(testVoice_path + '0/train/' + v)
    #voice가 male에 있으면
    else:
        tempVoice = os.listdir(testVoice_path + '1/train/' + v)
    #각 identity마다 face, voice 매치(combination)
    match_li = []
    for i in tempFace:
        for j in tempVoice:
            match = (i,j)
            match_li.append(match)
    align_result[f] = match_li
    
# Face - Voice 총 케이스 개수
count = 0
for i in align_result.keys():
    count += len(align_result[i])
    
print('총 케이스 개수:', count)

#pkl 파일로 만듬
with open('./FV/FValign/mask_train10.pkl', 'wb') as f:
    pickle.dump(align_result, f)

총 케이스 개수: 1377342


In [4]:
# test
testFace_path = './Face/DATA/sample10_5type/test/'
testVoice_path = './Voice/data/LibriSpeech/data_split/'

#매칭해준 사람에 맞춰 각 modality의 data align  
align_result = {}
for f,v in zip(Face2Voice, Face2Voice.values()):
    tempFace = os.listdir(testFace_path + f)
    #voice가 female에 있으면
    if v in os.listdir(testVoice_path + '0/test'):
        tempVoice = os.listdir(testVoice_path + '0/test/' + v)
    #voice가 male에 있으면
    else:
        tempVoice = os.listdir(testVoice_path + '1/test/' + v)
    #각 identity마다 face, voice 매치(combination)
    match_li = []
    for i in tempFace:
        for j in tempVoice:
            match = (i,j)
            match_li.append(match)
    align_result[f] = match_li
    
# Face - Voice 총 케이스 개수
count = 0
for i in align_result.keys():
    count += len(align_result[i])
    
print('총 케이스 개수:', count)

#pkl 파일로 만듬
with open('./FV/FValign/mask_test10.pkl', 'wb') as f:
    pickle.dump(align_result, f)

총 케이스 개수: 43536


In [5]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

with open('./FV/FValign/mask_train10.pkl','rb') as f:
    dict_ = pickle.load(f)
name = dict_.keys()

trainImg_path = []
trainVoice_path = []
for n in name:
    for i in range(len(dict_[n])):
        trainImg_path.append(n +'/'+dict_[n][i][0])
        voice_p = dict_[n][i][1].split('-')[0]
        if voice_p in os.listdir('./Voice/data/LibriSpeech/data_split/0/train'):
            trainVoice_path.append('0/train/'+voice_p+'/'+dict_[n][i][1])
        else:
            trainVoice_path.append('1/train/'+voice_p+'/'+dict_[n][i][1])

len(trainImg_path), len(trainVoice_path)

(1377342, 1377342)

In [6]:
with open('./FV/FValign/mask_test10.pkl','rb') as f:
    dict_ = pickle.load(f)
name = dict_.keys()

testImg_path = []
testVoice_path = []
for n in name:
    for i in range(len(dict_[n])):
        testImg_path.append(n +'/'+dict_[n][i][0])
        voice_p = dict_[n][i][1].split('-')[0]
        if voice_p in os.listdir('./Voice/data/LibriSpeech/data_split/0/test'):
            testVoice_path.append('0/test/'+voice_p+'/'+dict_[n][i][1])
        else:
            testVoice_path.append('1/test/'+voice_p+'/'+dict_[n][i][1])
            
len(testImg_path), len(testVoice_path)

(43536, 43536)

## loading images(facenet)

In [16]:
# train
label = []
file = []
for i in trainImg_path:
    label.append(i.split('/')[0])
    file.append(i.split('/')[-1])
    
image_train = pd.DataFrame()
image_train['file'] = file
image_train['label'] = label

facetrain_uni = set(trainImg_path)

label = []
file = []
for i in facetrain_uni:
    label.append(i.split('/')[0])
    file.append(i.split('/')[-1])
    
image_train2 = pd.DataFrame()
image_train2['file'] = file
image_train2['label'] = label

# test
label = []
file = []
for i in testImg_path:
    label.append(i.split('/')[0])
    file.append(i.split('/')[-1])
    
image_test = pd.DataFrame()
image_test['file'] = file
image_test['label'] = label

facetest_uni = set(testImg_path)

label = []
file = []
for i in facetest_uni:
    label.append(i.split('/')[0])
    file.append(i.split('/')[-1])
    
image_test2 = pd.DataFrame()
image_test2['file'] = file
image_test2['label'] = label

In [24]:
IMAGE_SIZE = (160, 160)
CHANNELS = 3

In [25]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

data_path = './Face/DATA/sample10_5type/train/'

image_w, image_h = IMAGE_SIZE

pixels = image_h * image_w * 3

X = []
for i in facetrain_uni:
    f = data_path + i
    img = Image.open(f)
    img = img.convert('RGB')
    img = img.resize((image_w,image_h))
    data = np.array(img)
    X.append(data)
        
train_X = np.array(X)

train_X.shape

(15996, 160, 160, 3)

In [26]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

data_path = './Face/DATA/sample10_5type/test/'

image_w, image_h = IMAGE_SIZE

pixels = image_h * image_w * 3

X = []
for i in facetest_uni:
    f = data_path + i
    img = Image.open(f)
    img = img.convert('RGB')
    img = img.resize((image_w,image_h))
    data = np.array(img)
    X.append(data)
        
test_X = np.array(X)

test_X.shape

(4026, 160, 160, 3)

In [27]:
# save arrays to one file in compressed format
np.savez_compressed('./FV/facenet_feature/10mask.npz', train_X, test_X)

In [28]:
import numpy as np
from keras.models import  load_model

# load the extracted faces dataset
data = np.load('./FV/facenet_feature/10mask.npz')
trainX, testX = data['arr_0'], data['arr_1']
print('Loaded: ', trainX.shape, testX.shape)

model = load_model('./Face/baseline/mtcnn2facenet/facenet_keras.h5')
print('Loaded Model')
model.summary()
# This specific implementation of the FaceNet model expects that the pixel values are standardized
# So, Standardizing the inputs
# scale pixel values
def get_embedding(model,face_pixels):
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std

    # To make predictions with FaceNet input vector needs to be in one sample 
    # transform face into one sample
    samples = np.expand_dims(face_pixels, axis=0)

    # using model predictions to get face embedding for the images
    yhat = model.predict(samples)
    embedding = yhat[0]
    return embedding

newtrainX = list()
newtestX = list()

for face_pixels in trainX:
    embedding = get_embedding(model,face_pixels)
    newtrainX.append(embedding)
newtrainX = np.asarray(newtrainX)

for face_pixels in testX:
    embedding = get_embedding(model,face_pixels)
    newtestX.append(embedding)
newtestX = np.asarray(newtestX)

np.savez_compressed('./FV/facenet_feature/10maskFacenet.npz',newtrainX,newtestX)

Loaded:  (15996, 160, 160, 3) (4026, 160, 160, 3)
Loaded Model
Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
_________________

In [18]:
# set에 대해서 이미지 로드
# facenet 임베딩
# align 에 맞춰서 피처 저장
# 보이스도 똑같이 해주고 학습

In [29]:
data = np.load('./FV/facenet_feature/10maskFacenet.npz')
facenet_train, facenet_test = data['arr_0'], data['arr_1']

In [35]:
facenet_train = facenet_train.tolist()
facenet_test = facenet_test.tolist()

In [36]:
image_train2['feature'] = facenet_train
train_feature = []
for i in image_train.file:
    train_feature.append(image_train2[image_train2.file == i].feature.tolist()[0])
    
image_train['feature'] = train_feature

image_test2['feature'] = facenet_test
test_feature = []
for i in image_test.file:
    test_feature.append(image_test2[image_test2.file == i].feature.tolist()[0])
    
image_test['feature'] = test_feature

np.savez_compressed('./FV/facenet_feature/FV_10maskFacenet.npz',train_feature,test_feature)

In [52]:
len(image_train)

1377342

## loading voice

In [38]:
# train
label = []
file = []
for i in trainVoice_path:
    label.append(i.split('/')[0])
    file.append(i.split('/')[-1])
    
voice_train = pd.DataFrame()
voice_train['file'] = file
voice_train['label'] = label

vt_uni = set(trainVoice_path)

label = []
file = []
for i in vt_uni:
    label.append(i.split('/')[0])
    file.append(i.split('/')[-1])
    
voice_train2 = pd.DataFrame()
voice_train2['file'] = file
voice_train2['label'] = label

# Although this function was modified and many parameteres were explored with, most of it
# came from Source 8 (sources in the READ.ME)

def extract_features(files):
    
    # Sets the name to be the path to where the file is in my computer
#     file_name = os.path.join(os.path.abspath('voice')+'/'+str(files.file))
    print('files:', files)
    print('label:', files.label)
    dir = './Voice/data/LibriSpeech/data_split/'
    file_name = os.path.join( dir + files.label +'/train/'+files.file.split('-')[0] +'/'+str(files.file))

    #파일이 아닌 디렉토리를 열어서 에러남 
    
    
    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    # We add also the classes of each file as a label at the end
    label = files.label

    return mfccs, chroma, mel, contrast, tonnetz, label

In [39]:
# Applying the function to the train data by accessing each row of the dataframe
from datetime import datetime
import librosa.display

startTime = datetime.now()

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

features_label_train = voice_train2.apply(extract_features, axis=1)

print(datetime.now() - startTime)

features_train = []
for i in range(0, len(features_label_train)):
    features_train.append(np.concatenate((features_label_train[i][0], features_label_train[i][1], 
                features_label_train[i][2], features_label_train[i][3],
                features_label_train[i][4]), axis=0))
    
voice_train2['feature'] = features_train
train_feature = []
for i in voice_train.file:
    train_feature.append(voice_train2[voice_train2.file == i].feature.tolist()[0])
    
voice_train['feature'] = train_feature

files: file     103-1240-0016.flac
label                     0
Name: 0, dtype: object
label: 0
files: file     39-121916-0009.flac
label                      0
Name: 1, dtype: object
label: 0
files: file     4340-15220-0095.flac
label                       0
Name: 2, dtype: object
label: 0
files: file     4853-27671-0035.flac
label                       0
Name: 3, dtype: object
label: 0
files: file     4853-27670-0031.flac
label                       0
Name: 4, dtype: object
label: 0
files: file     1334-135589-0092.flac
label                        1
Name: 5, dtype: object
label: 1
files: file     2764-36616-0035.flac
label                       0
Name: 6, dtype: object
label: 0
files: file     103-1240-0007.flac
label                     0
Name: 7, dtype: object
label: 0
files: file     4340-15220-0022.flac
label                       0
Name: 8, dtype: object
label: 0
files: file     4340-15220-0058.flac
label                       0
Name: 9, dtype: object
label: 0
files: file     84

files: file     39-121915-0003.flac
label                      0
Name: 83, dtype: object
label: 0
files: file     4853-27670-0026.flac
label                       0
Name: 84, dtype: object
label: 0
files: file     1334-135589-0090.flac
label                        1
Name: 85, dtype: object
label: 1
files: file     8425-291444-0016.flac
label                        1
Name: 86, dtype: object
label: 1
files: file     39-121916-0018.flac
label                      0
Name: 87, dtype: object
label: 0
files: file     103-1240-0045.flac
label                     0
Name: 88, dtype: object
label: 0
files: file     4853-27671-0000.flac
label                       0
Name: 89, dtype: object
label: 0
files: file     4853-27671-0030.flac
label                       0
Name: 90, dtype: object
label: 0
files: file     4340-15220-0038.flac
label                       0
Name: 91, dtype: object
label: 0
files: file     39-121916-0028.flac
label                      0
Name: 92, dtype: object
label: 0
files:

files: file     1334-135589-0028.flac
label                        1
Name: 165, dtype: object
label: 1
files: file     4853-27671-0016.flac
label                       0
Name: 166, dtype: object
label: 0
files: file     7113-86041-0027.flac
label                       0
Name: 167, dtype: object
label: 0
files: file     4340-15220-0047.flac
label                       0
Name: 168, dtype: object
label: 0
files: file     4340-15220-0078.flac
label                       0
Name: 169, dtype: object
label: 0
files: file     8425-287387-0021.flac
label                        1
Name: 170, dtype: object
label: 1
files: file     4853-27671-0037.flac
label                       0
Name: 171, dtype: object
label: 0
files: file     39-121916-0006.flac
label                      0
Name: 172, dtype: object
label: 0
files: file     39-121916-0022.flac
label                      0
Name: 173, dtype: object
label: 0
files: file     6209-34599-0018.flac
label                       1
Name: 174, dtype: object

files: file     2764-36619-0029.flac
label                       0
Name: 247, dtype: object
label: 0
files: file     4340-15220-0052.flac
label                       0
Name: 248, dtype: object
label: 0
files: file     4853-27670-0029.flac
label                       0
Name: 249, dtype: object
label: 0
files: file     289-121652-0003.flac
label                       0
Name: 250, dtype: object
label: 0
files: file     2764-36616-0002.flac
label                       0
Name: 251, dtype: object
label: 0
files: file     2764-36619-0018.flac
label                       0
Name: 252, dtype: object
label: 0
files: file     1334-135589-0052.flac
label                        1
Name: 253, dtype: object
label: 1
files: file     8425-291444-0017.flac
label                        1
Name: 254, dtype: object
label: 1
files: file     289-121665-0011.flac
label                       0
Name: 255, dtype: object
label: 0
files: file     6209-34599-0032.flac
label                       1
Name: 256, dtype: ob

files: file     4853-27670-0027.flac
label                       0
Name: 329, dtype: object
label: 0
files: file     2764-36617-0003.flac
label                       0
Name: 330, dtype: object
label: 0
files: file     7113-86041-0019.flac
label                       0
Name: 331, dtype: object
label: 0
files: file     103-1240-0017.flac
label                     0
Name: 332, dtype: object
label: 0
files: file     289-121665-0009.flac
label                       0
Name: 333, dtype: object
label: 0
files: file     103-1240-0029.flac
label                     0
Name: 334, dtype: object
label: 0
files: file     7113-86041-0052.flac
label                       0
Name: 335, dtype: object
label: 0
files: file     7113-86041-0057.flac
label                       0
Name: 336, dtype: object
label: 0
files: file     8425-287387-0027.flac
label                        1
Name: 337, dtype: object
label: 1
files: file     4853-27670-0009.flac
label                       0
Name: 338, dtype: object
label

...

files: file     4340-15220-0030.flac
label                       0
Name: 493, dtype: object
label: 0
files: file     1334-135589-0054.flac
label                        1
Name: 494, dtype: object
label: 1
files: file     289-121665-0039.flac
label                       0
Name: 495, dtype: object
label: 0
files: file     4340-15220-0070.flac
label                       0
Name: 496, dtype: object
label: 0
files: file     6209-34599-0009.flac
label                       1
Name: 497, dtype: object
label: 1
files: file     6209-34601-0027.flac
label                       1
Name: 498, dtype: object
label: 1
files: file     1334-135589-0063.flac
label                        1
Name: 499, dtype: object
label: 1
files: file     1334-135589-0023.flac
label                        1
Name: 500, dtype: object
label: 1
files: file     2764-36619-0016.flac
label                       0
Name: 501, dtype: object
label: 0
files: file     103-1240-0057.flac
label                     0
Name: 502, dtype: obje

files: file     4340-15220-0068.flac
label                       0
Name: 575, dtype: object
label: 0
files: file     6209-34601-0002.flac
label                       1
Name: 576, dtype: object
label: 1
files: file     4853-27671-0020.flac
label                       0
Name: 577, dtype: object
label: 0
files: file     39-121916-0030.flac
label                      0
Name: 578, dtype: object
label: 0
files: file     8425-287387-0017.flac
label                        1
Name: 579, dtype: object
label: 1
files: file     289-121665-0046.flac
label                       0
Name: 580, dtype: object
label: 0
files: file     8425-246962-0025.flac
label                        1
Name: 581, dtype: object
label: 1
files: file     1334-135589-0026.flac
label                        1
Name: 582, dtype: object
label: 1
files: file     1334-135589-0018.flac
label                        1
Name: 583, dtype: object
label: 1
files: file     103-1241-0033.flac
label                     0
Name: 584, dtype: obje

files: file     7113-86041-0068.flac
label                       0
Name: 657, dtype: object
label: 0
files: file     6209-34601-0006.flac
label                       1
Name: 658, dtype: object
label: 1
files: file     4340-15220-0090.flac
label                       0
Name: 659, dtype: object
label: 0
files: file     1334-135589-0050.flac
label                        1
Name: 660, dtype: object
label: 1
files: file     289-121652-0009.flac
label                       0
Name: 661, dtype: object
label: 0
files: file     4853-29413-0022.flac
label                       0
Name: 662, dtype: object
label: 0
files: file     8425-287387-0023.flac
label                        1
Name: 663, dtype: object
label: 1
files: file     6209-34599-0001.flac
label                       1
Name: 664, dtype: object
label: 1
files: file     289-121652-0029.flac
label                       0
Name: 665, dtype: object
label: 0
files: file     6209-34599-0029.flac
label                       1
Name: 666, dtype: ob

files: file     2764-36616-0034.flac
label                       0
Name: 739, dtype: object
label: 0
files: file     8425-246962-0012.flac
label                        1
Name: 740, dtype: object
label: 1
files: file     6209-34601-0001.flac
label                       1
Name: 741, dtype: object
label: 1
files: file     1334-135589-0021.flac
label                        1
Name: 742, dtype: object
label: 1
files: file     103-1240-0041.flac
label                     0
Name: 743, dtype: object
label: 0
files: file     4340-15220-0073.flac
label                       0
Name: 744, dtype: object
label: 0
files: file     1334-135589-0079.flac
label                        1
Name: 745, dtype: object
label: 1
files: file     289-121652-0036.flac
label                       0
Name: 746, dtype: object
label: 0
files: file     2764-36616-0027.flac
label                       0
Name: 747, dtype: object
label: 0
files: file     103-1241-0027.flac
label                     0
Name: 748, dtype: object
l

files: file     4340-15220-0024.flac
label                       0
Name: 821, dtype: object
label: 0
files: file     6209-34601-0016.flac
label                       1
Name: 822, dtype: object
label: 1
files: file     289-121652-0026.flac
label                       0
Name: 823, dtype: object
label: 0
files: file     4340-15220-0059.flac
label                       0
Name: 824, dtype: object
label: 0
files: file     39-121914-0000.flac
label                      0
Name: 825, dtype: object
label: 0
files: file     8425-246962-0006.flac
label                        1
Name: 826, dtype: object
label: 1
files: file     289-121665-0010.flac
label                       0
Name: 827, dtype: object
label: 0
files: file     39-121914-0019.flac
label                      0
Name: 828, dtype: object
label: 0
files: file     7113-86041-0053.flac
label                       0
Name: 829, dtype: object
label: 0
files: file     8425-287387-0016.flac
label                        1
Name: 830, dtype: object

In [40]:
# test
label = []
file = []
for i in testVoice_path:
    label.append(i.split('/')[0])
    file.append(i.split('/')[-1])
    
voice_test = pd.DataFrame()
voice_test['file'] = file
voice_test['label'] = label

vt_uni = set(testVoice_path)

label = []
file = []
for i in vt_uni:
    label.append(i.split('/')[0])
    file.append(i.split('/')[-1])
    
voice_test2 = pd.DataFrame()
voice_test2['file'] = file
voice_test2['label'] = label

# Although this function was modified and many parameteres were explored with, most of it
# came from Source 8 (sources in the READ.ME)

def extract_features(files):
    
    # Sets the name to be the path to where the file is in my computer
#     file_name = os.path.join(os.path.abspath('voice')+'/'+str(files.file))
    print('files:', files)
    print('label:', files.label)
    dir = './Voice/data/LibriSpeech/data_split/'
    file_name = os.path.join( dir + files.label +'/test/'+files.file.split('-')[0] +'/'+str(files.file))

    #파일이 아닌 디렉토리를 열어서 에러남 
    
    
    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    # We add also the classes of each file as a label at the end
    label = files.label

    return mfccs, chroma, mel, contrast, tonnetz, label

In [41]:
# Applying the function to the train data by accessing each row of the dataframe
from datetime import datetime
import librosa.display

startTime = datetime.now()

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

features_label_test = voice_test2.apply(extract_features, axis=1)

print(datetime.now() - startTime)

features_test = []
for i in range(0, len(features_label_test)):
    features_test.append(np.concatenate((features_label_test[i][0], features_label_test[i][1], 
                features_label_test[i][2], features_label_test[i][3],
                features_label_test[i][4]), axis=0))
    
voice_test2['feature'] = features_test
test_feature = []
for i in voice_test.file:
    test_feature.append(voice_test2[voice_test2.file == i].feature.tolist()[0])
    
voice_test['feature'] = test_feature

files: file     289-121652-0028.flac
label                       0
Name: 0, dtype: object
label: 0
files: file     1334-135589-0006.flac
label                        1
Name: 1, dtype: object
label: 1
files: file     1334-135589-0056.flac
label                        1
Name: 2, dtype: object
label: 1
files: file     103-1241-0017.flac
label                     0
Name: 3, dtype: object
label: 0
files: file     103-1240-0032.flac
label                     0
Name: 4, dtype: object
label: 0
files: file     289-121652-0014.flac
label                       0
Name: 5, dtype: object
label: 0
files: file     6209-34599-0031.flac
label                       1
Name: 6, dtype: object
label: 1
files: file     1334-135589-0036.flac
label                        1
Name: 7, dtype: object
label: 1
files: file     8425-287387-0003.flac
label                        1
Name: 8, dtype: object
label: 1
files: file     39-121914-0008.flac
label                      0
Name: 9, dtype: object
label: 0
files: file 

files: file     6209-34601-0013.flac
label                       1
Name: 83, dtype: object
label: 1
files: file     2764-36617-0013.flac
label                       0
Name: 84, dtype: object
label: 0
files: file     6209-34600-0003.flac
label                       1
Name: 85, dtype: object
label: 1
files: file     2764-36619-0041.flac
label                       0
Name: 86, dtype: object
label: 0
files: file     2764-36619-0012.flac
label                       0
Name: 87, dtype: object
label: 0
files: file     39-121915-0005.flac
label                      0
Name: 88, dtype: object
label: 0
files: file     4853-27670-0028.flac
label                       0
Name: 89, dtype: object
label: 0
files: file     8425-287387-0032.flac
label                        1
Name: 90, dtype: object
label: 1
files: file     6209-34601-0023.flac
label                       1
Name: 91, dtype: object
label: 1
files: file     2764-36619-0017.flac
label                       0
Name: 92, dtype: object
label: 0


In [42]:
np.savez_compressed('./FV/voice_feature/FV_10maskvoice.npz',train_feature,test_feature)

## loading demography

In [71]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

train_demo = pd.read_csv('./Face/DATA/demography/train10_originaldemo_0228.csv')

train_demo['gender_'] = LabelEncoder().fit_transform(train_demo['gender'])
train_demo['race_'] = LabelEncoder().fit_transform(train_demo['race'])
train_demo['emotion_'] = LabelEncoder().fit_transform(train_demo['emotion'])

test_demo = pd.read_csv('./Face/DATA/demography/test10_originaldemo_0228.csv')

test_demo['gender_'] = LabelEncoder().fit_transform(test_demo['gender'])
test_demo['race_'] = LabelEncoder().fit_transform(test_demo['race'])
test_demo['emotion_'] = LabelEncoder().fit_transform(test_demo['emotion'])

In [83]:
age = []
gender = []
race = []

for i in range(len(image_train)):
    if len(image_train.file[i].split('_')) == 3:
        id_ = image_train.label[i] + '/' + image_train.file[i].split('_')[0]+'_'+image_train.file[i].split('_')[1]
        age.append(train_demo[train_demo.id == id_].age.tolist()[0])
        gender.append(train_demo[train_demo.id == id_].gender_.tolist()[0])
        race.append(train_demo[train_demo.id == id_].race_.tolist()[0])
    else:
        id_ = image_train.label[i] + '/' + image_train.file[i].split('.jpg')[0]
        age.append(train_demo[train_demo.id == id_].age.tolist()[0])
        gender.append(train_demo[train_demo.id == id_].gender_.tolist()[0])
        race.append(train_demo[train_demo.id == id_].race_.tolist()[0])
               
image_train['age'] = age
image_train['gender'] = gender
image_train['race'] = race

age = []
gender = []
race = []

for i in range(len(image_test)):
    if len(image_test.file[i].split('_')) == 3:
        id_ = image_test.label[i] + '/' + image_test.file[i].split('_')[0]+'_'+image_test.file[i].split('_')[1]
        age.append(test_demo[test_demo.id == id_].age.tolist()[0])
        gender.append(test_demo[test_demo.id == id_].gender_.tolist()[0])
        race.append(test_demo[test_demo.id == id_].race_.tolist()[0])
    else:
        id_ = image_test.label[i] + '/' + image_test.file[i].split('.jpg')[0]
        age.append(test_demo[test_demo.id == id_].age.tolist()[0])
        gender.append(test_demo[test_demo.id == id_].gender_.tolist()[0])
        race.append(test_demo[test_demo.id == id_].race_.tolist()[0])
               
image_test['age'] = age
image_test['gender'] = gender
image_test['race'] = race

In [84]:
trainDemo_feature = []
for i in range(len(image_train)):
    temp = np.array([image_train.age[i], image_train.gender[i], image_train.race[i]])
    trainDemo_feature.append(temp)

testDemo_feature = []
for i in range(len(image_test)):
    temp = np.array([image_test.age[i], image_test.gender[i], image_test.race[i]])
    testDemo_feature.append(temp)

trainDemo_feature = np.array(trainDemo_feature)
testDemo_feature = np.array(testDemo_feature)

trainDemo_feature.shape, testDemo_feature.shape

((1377342, 3), (43536, 3))

In [85]:
np.savez_compressed('./FV/demography/FV_10maskdemo.npz',trainDemo_feature,testDemo_feature)

## model

In [80]:
import numpy as np
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
from sklearn.preprocessing import OneHotEncoder

# load face embeddings
facedata = np.load('./FV/facenet_feature/FV_10maskFacenet.npz')
facenet_train, facenet_test = facedata['arr_0'], facedata['arr_1']
# normalize input vectors
input_encoder = Normalizer(norm='l2')
facenet_train = input_encoder.fit_transform(facenet_train)
facenet_test = input_encoder.transform(facenet_test)
# label encode targets
trainy = image_train.label.values
testy = image_test.label.values

output_encoder = OneHotEncoder(sparse=False)

trainy = trainy.reshape(-1,1)
testy = testy.reshape(-1,1)

trainy = output_encoder.fit_transform(trainy)
testy = output_encoder.transform(testy)
#load voice embeddings
voicedata = np.load('./FV/voice_feature/FV_10maskvoice.npz')
voice_train, voice_test = voicedata['arr_0'], voicedata['arr_1']

facenet_train.shape, voice_train.shape, trainy.shape, facenet_test.shape, voice_test.shape, testy.shape

((1377342, 128),
 (1377342, 193),
 (1377342, 10),
 (43536, 128),
 (43536, 193),
 (43536, 10))

In [81]:
import os
import pandas as pd
import numpy as np
import glob
from PIL import Image

import matplotlib.pyplot as plt

import json
from deepface.extendedmodels import Age, Gender, Race, Emotion
from deepface import DeepFace

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow import keras
from os import path
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
import sys

In [89]:
SHUFFLE_BUFFER = 1 # set this externally to the number of items in dataset for ideal results
NUM_CLASSES = 10 ####change your classes num
BATCH_SIZE = 4
LEARN_RATE = 0.01 * (BATCH_SIZE / 128)
MOMENTUM = 0.9
EPOCHS = 1
embedding_size = 128

In [90]:
from keras.models import Model
import keras.backend as K
from keras.layers import Input, Reshape, Dense, Embedding, Dropout, LSTM, Lambda, Concatenate, \
    Multiply, RepeatVector, Permute, Flatten, Activation

def Multimodal(image_size=IMAGE_SIZE, channels=CHANNELS, num_classes=NUM_CLASSES, learn_rate=LEARN_RATE, momentum=MOMENTUM):
    

    inputs_facefeature = Input(shape=(128,))
    inputs_voice = Input(shape=(193,))
    inputs_demo = Input(shape=(3,))
    
### image
    wt_init = keras.initializers.RandomNormal(mean=0, stddev=0.01)
    bias_init = keras.initializers.Constant(value=0.5)
    
    def dense_layer(**args):
        return keras.layers.Dense(**args, 
            kernel_initializer=wt_init, 
            bias_initializer=bias_init)
    
### voice
    V1 = Dense(193, activation='relu')(inputs_voice)
    D1 = keras.layers.Dropout(rate=0.1, name='D1')(V1)
    V2 = Dense(128, activation='relu')(D1)
    D2 = keras.layers.Dropout(rate=0.25, name='D2')(V2)
    V3 = Dense(128, activation='relu')(D1)
    D3 = keras.layers.Dropout(rate=0.5, name='D3')(V3)

### demo    
    # demoFeature layer 추가
    demoFeature = Dense(embedding_size, activation='tanh')(inputs_demo)
    
    # demoFeature 랑 face feature similarity 계산
    sim = Multiply()([inputs_facefeature, demoFeature])
    att = Dense(1, activation='tanh')(sim)
    att = Flatten()(att)
    
    # compute the weights of eache demographic feature
    attention = Activation('softmax')(att)
    attention = RepeatVector(embedding_size)(attention) #embedding_size
    
    # similarity랑 weights 곱하기(weighted sum)
    influence = Multiply()([sim, attention])
    influence = Lambda(lambda x: K.sum(x, axis=1))(influence)
    
    # the most important demographic feature of classification face
    influence = Dense(embedding_size)(influence)
    
    h = Concatenate()([inputs_facefeature, D3, influence])

    Softmax = dense_layer(units=num_classes, activation=keras.activations.softmax, name='F8')(h)

    model = Model(inputs=[inputs_facefeature, inputs_voice, inputs_demo],
                      outputs=[Softmax])

    sgd_opt = keras.optimizers.SGD(learning_rate=LEARN_RATE, momentum=MOMENTUM)
    cce_loss = keras.losses.categorical_crossentropy
    
    model.compile(optimizer=sgd_opt, loss=cce_loss, metrics=['accuracy'])
    
    return model

In [92]:
model1 = Multimodal()

es = EarlyStopping(monitor='loss', verbose=1, min_delta=0, mode='auto', patience= 5)
mc = ModelCheckpoint('./FV/10maskdemo_FV.h5', monitor='loss', save_best_only=True)

model1.fit([facenet_train,voice_train,trainDemo_feature],trainy, steps_per_epoch = facenet_train.shape[0] // BATCH_SIZE, \
           epochs = EPOCHS, callbacks = [es,mc])

# model1.fit(facenet_trainX,facenet_trainy, steps_per_epoch = facenet_trainX.shape[0] // BATCH_SIZE, \
#            epochs = EPOCHS, callbacks = [es,mc])

344335/344335 [==============================] - 233s 675us/step - loss: 0.2831 - accuracy: 0.9053


In [93]:
from keras.models import load_model
savedmodel = load_model('./FV/10maskdemo_FV.h5')

score = savedmodel.evaluate([facenet_test,voice_test,testDemo_feature],testy)
# score = savedmodel.evaluate(facenet_testX,facenet_testy)

1361/1361 [==============================] - 1s 672us/step - loss: 0.0360 - accuracy: 0.9900
